In [ ]:
#########################################################################
# Video Classification Tutorial - Using Inception v3 CNN + RNN & LSTM
# TensorFlow CPU + Keras 
#########################################################################


# GOAL - To understand the actions within a small video file (3-10 sec) 
# and then make a prediction of this video into various predefined classes. 


# DETAILS: The classes are embeded directly into the file name 
# and folder structure and will be substracted.

# From what I understand, we will first prepare the CNN by downloading 
# the Inceptionv3 pretrained algorithm (in TensorFlow and Keras) 
# and then add our specific video information into the top layer 
# of Inceptionv3 and retrain just that portion of the neural network.

# We will then flatten the results of the CNN and then feed the newly saved 
# features/weights into a Long Short Term Memory(LSTM) Recurrent Neural 
# Network (RNN) for further processing.

# What this code should do would be to take the video and split it into
# individual images via the cv2 package. For ease of use, we grab the 25th 
# frame of each video and then feed this into the the CNN.

# I think that the goal will be to seperate this code into at least 2 files
#      1. Training & Testing Procedure
#      2. Application of trained algorithm to new video


#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# This is the video instructions of the author and coders work 
# (Gokhan Cagrici) which we are trying to replicate and recreate. 
# This is the most valuable resource that we have (and all that
# we have, unfortunately)
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# https://www.youtube.com/watch?v=Txbfx47pYtg

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# This is the location of the data sources and background information:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# http://crcv.ucf.edu/data/UCF101.php#Results_on_UCF101
# http://www.thumos.info/download.html

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# IMPORTANT NOTE
# The code is fairly complete but still needs to be audited cell by cell 
# and carefully troubleshooted. I think it is about 95%-98% complete.

# However, I am certain there are small missing pieces & code snippets 
# and we will have to make the corrections ourselves at each step. 

# Can you figure out how to solve the missing pieces to this puzzle?

In [1]:
import keras
import numpy as np
import matplotlib.pyplot as plt

C:\Users\derek\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras.applications import InceptionV3

In [3]:
# We will re-train the top layer of Inception with our own dataset. Thats why we define include_top to false
# The shape of the individual image is 3 layers of 240 pixels by 320 pixels

conv_base = InceptionV3(weights='imagenet',
                       include_top=False,
                       input_shape=(240, 320, 3)
                       )

In [4]:
conv_base.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 240, 320, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 119, 159, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 119, 159, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 119, 159, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [11]:
def videos2frames(source_dir, target_dir, train_percentage):
    import os
    import re
    
    delete_dir(target_dir)
    create_dir_if_not_exists(target_dir)
    
    data_dirs = (f for f in sorted(os.listdir(source_dir)) if not f.startswith('.'))
                
    for cat_name in data_dirs:
        source_cat_dir = os.path.join(source_dir, cat_name)
        #### Line where I made changes #####
        groups = list({re.search('^\w+_\w+_g(\d\d)_c(\d\d)', f).group(1) for f in os.listdir(source_cat_dir) if not f.startswith('.')})
        group_size = len(groups)
        np.random.shuffle(groups)
                 
        x = 0
        y = 0
        
        video_dir = os.path.join(source_dir, cat_name)
        
        video_path = (f for f in os.listdir(video_dir) if not f.startswith('.'))
        for video_file in video_path:
        #for video_file in sorted(os.listdir(video_dir))[1:]:
        #for video_file in video_dir:
            m = re.search('^\w+_\w+_g(\d\d)_c(\d\d)', video_file)
            group_name = m.group(1)
            subject_name = m.group(2)
                 
            if group_name in groups[:int(group_size * train_percentage)]:
                parent_dir = 'train'
                x += 1
            else:
                parent_dir = 'test'
                y += 1
                 
            src_video_path = os.path.join(video_dir, video_file)
            frames_dir = os.path.join(os.path.join(target_dir, parent_dir),cat_name)
                 
            create_dir_if_not_exists(frames_dir)
            extract_frames_from_video(src_video_path, frames_dir, "{}-{}".format(group_name, subject_name))
                
def create_dir_if_not_exists(path):
    import os
    if not os.path.exists(path):
        os.makedirs(path)
                        
def delete_dir(path):
    import shutil
    import os
                 
    if os.path.exists(path):
        shutil.rmtree(path)

In [12]:
def extract_frames_from_video(video_path, target_dir, target_file_prefix):
    import cv2
    
    v = cv2.VideoCapture(video_path)
    
    success, image = v.read()
    count = 0 
    
    while success:
        count += 1
        if count % 25 == 0:
            cv2.imwrite('{}/{}-{}.jpg'.format(target_dir, target_file_prefix, count), image)
            #cv2.imwrite('{}/{}-{}.jpg'.format(target_dir, target_file_prefix))
        success, image = v.read()

In [13]:
videos2frames('C:/Users/derek/Documents/Python_Scripts/Project_Adam/Raw_Video/', 'C:/Users/derek/Documents/Python_Scripts/Project_Adam/Raw_Video/UCF-101-POC-Frames', 0.95)

AttributeError: 'NoneType' object has no attribute 'groups'

In [ ]:
import os

base_dir = '/home/paperspace/Advanced-Video-Classification/UCF-101-POC-Frames'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'test')
batch_size = 64

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 6, 8, 2048))
    labels = np.zeros(shape=(sample_count, 3))
    generator = datagen.flow_from_directory(
        directory,
        target_size=(240, 320),
        batch_size=batch_size,
        class_mode='categorical')
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        print(i * batch_size)
        if i * batch_size >= sample_count:
            break
    return features, labels 

In [ ]:
generator = datagen.flow_from_directory(validation_dir, 
                                        target_size=(240, 320), 
                                        batch_size=batch_size, 
                                        class_mode='categorical')

In [ ]:
generator_train = datagen.flow_from_directory(train_dir, 
                                              target_size=(240, 320), 
                                              batch_size=batch_size, 
                                              class_mode='categorical')

In [ ]:
train_features, train_labels = extract_features(train_dir, 3104)

In [ ]:
np.save("train_features.npy", train_features)
np.save("train_labels.npy", train_labels)
train_features.shape, train_labels.shape

In [ ]:
# This code needs to be checked. I could not see all of it in the video.

validation_features, validation_labels = extract_features(validation_dir, 237)

In [ ]:
np.save("validation_features.npy", validation_features)
np.save("validation_labels.npy", validation_labels)
validation_features.shape, validation_labels.shape

In [ ]:
# 85164 sample files in the training set, 7725 in the validation set. 

train_features = np.reshape(train_features, (3104, 6 * 8 * 2048))
validation_features = np.reshape(validation_features, (237, 6 * 8 * 2048))

In [ ]:
from keras import models
from keras import layers
from keras import optimizers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim=6 * 8 * 2048))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(3, activation='softmax'))

model.compile(optimizer=optimizers.adam(lr=2e-6),
             loss='categorical_crossentropy',
             metrics=['acc'])

history = model.fit(train_features, train_labels,
                   epochs=10,
                   batch_size=64,
                   validation_data=(validation_features, validation_labels))

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.figure()

plt.show()

In [ ]:
model.save('inception3.h5')

In [ ]:
# There might be some code here too. Cant see in the video. Sorry.



In [ ]:
train_dir, validation_dir

In [ ]:
def find_cat_groups(dir_name):
    import re
    
    cat_groups = {}
    
    for cat in os.listdir(dir_name):
        samples_dir = os.path.join(dir_name, cat)
        cat_groups[cat] = {re.search('^(\d+)', f).group(1) for f in os.listdir(samples_dir) if not f.startswith('.')}
    
    return cat_groups

In [ ]:
# This code needs to be checked. I could not see all of it in the video.

cat_groups_train, cat_groups_validation = find_cat_groups(train_dir), find_cat_groups(validation_dir)

In [ ]:
cat_groups_train

In [ ]:
cat_groups_validation

In [ ]:
len(cat_groups_train), len(cat_groups_validation)

In [ ]:
# This code needs to be checked. I could not see all of it in the video.

cat_dict = {cat_name: i for i, cat_name in enumerate(cat_groups_train.keys())}

In [ ]:
from keras import models
from keras import layers
from keras import optimizers

model = model.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim=6 * 8 * 2048))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='softmax'))

In [ ]:
print('BaseballPitch', cat_dict['BaseballPitch'])

In [ ]:
from keras import models
from keras import layers
from keras import optimizers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim=6 * 8 * 2048))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(3, activation='softmax'))

In [ ]:
model.load_weights('inception3.h5')

In [ ]:
model.summary()

In [ ]:
from keras import models
from keras import layers

model2 = models.Sequential()
model2.add(conv_base)
model2.add(layers.Flatten())
model2.add(model.layers[0])

In [ ]:
model2.summary()

In [ ]:
def extract_features_RNN(cat_dict, dir_path, cat_groups, sample_size):
    import re
    import cv2
    
    s = 0
    
    features = np.zeros(shape=(sample_size, 80 , 256))
    labels = np.zeros(shape=(sample_size, 3))
    frames = np.zeros(shape=(80, 240, 320, 3))
    
    video_path = (f for f in os.listdir(dir_path) if not f.startswith('.'))
    for cat in video_path:
    #for cat in sorted(os.listdir(dir_path))[1:]:
        print(cat, cat_dict[cat])
        cat_dir = os.path.join(dir_path, cat)
        video_files = (f for f in os.listdir(cat_dir) if not f.startswith('.'))
        for f in video_files:
        #for f in sorted(video_files)[1:]:
            if s >= sample_size:
                break
            group_name = re.search('^\w+_\w+_g(\d\d)', f).group(1)
            if group_name in cat_groups[cat]:
                v = cv2.VideoCapture(os.path.join(cat_dir, f))
                frame_count = int(v.get(cv2.CAP_PROP_FRAME_COUNT))
                frame_height = int(v.get(cv2.CAP_PROP_FRAME_HEIGHT))
                frame_width = int(v.get(cv2.CAP_PROP_FRAME_WIDTH))
                if frame_count >= 80 and frame_height == 240 and frame_width == 320:
                    read_frames_idx = list(range(0, frame_count, frame_count))
                    for i, idx in enumerate(read_frames_idx):
                        v.set(cv2.CAP_PROP_POS_FRAMES, idx)
                        success, image = v.read()
                            #print(s, i, idx, success)
                        if success:
                            frames[i] = image / 255.
                        else:
                            print("ERROR")
                    features[s, :, :] = model2.predict(frames)
                    labels[s, cat_dict[cat]] = 1
                    print(s)
                    s += 1
                        
    return s, features, labels

In [ ]:
size, features_rnn, labels_rnn = extract_features_RNN(cat_dict, '/home/paperspace/Advanced-Video-Classification/UCF-101/', cat_groups_train, 3104)

In [ ]:
np.save("features_rnn", features_rnn[:size,:,:])
np.save("labels_rnn", labels_rnn[:size,:])
size

In [ ]:
size_v, features_rnn_v, labels_rnn_v = extract_features_RNN(cat_dict, '/home/paperspace/Advanced-Video-Classification/UCF-101/', cat_groups_validation, 237)

In [ ]:
np.save("features_rnn_v.np", features_rnn_v[:size_v,:,:])
np.save("labels_rnn_v.np", labels_rnn_v[:size_v,:])

In [ ]:
features_rnn = np.load("features_rnn.np.npy")
labels_rnn = np.load("labels_rnn.np.npy")

In [ ]:
features_rnn.shape, labels_rnn.shape

In [ ]:
features_rnn_v = np.load("features_rnn_v.np.npy")
labels_rnn_v = np.load("labels_rnn_v.np.npy")

In [ ]:
features_rnn_v.shape, labels_rnn_v.shape

In [ ]:
# This code needs to be checked. I could not see all of it in the video.

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

model = Sequential()
model.add(LSTM(200, input_shape=(80, 256)))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['acc'])

In [ ]:
hist = model.fit(features_rnn, labels_rnn,
                epochs=30,
                batch_size=64,
                validation_data=(features_rnn_v, labels_rnn_v))

In [ ]:
acc = hist.history['acc']
val_acc = hist.history['val_acc']
loss = hist.history['loss']
val_loss = hist.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.figure()

plt.show()